In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:
# 한 건의 대화 정보를 저장하는 클래스
class Conversation:
    # 생성자 => 클래스의 멤버 변수(self가 붙은 변수)를 초기화 시킨다.
    # 클래스 내부에 정의하는 멤버 함수는 첫 번째 인수로 자기 자신의 객체가 넘어온다. => 첫 번째
    # 인수는 무조건 self를 써야한다.
    def __init__(self, question, answer):
        # 생성자의 인수로 넘겨받은 데이터로 멤버 변수를 초기화 시킨다.
        # 멤버 변수는 자기 자신의 객체를 저장한 인수인 self를 사용해서 선언해야 한다.
        self.question = question # 질문을 기억하는 멤버 변수를 선언하고 초기화 한다.
        self.answer = answer     # 답변을 기억하는 멤버 변수를 선언하고 초기화 한다.
    # 클래스로 생성한 객체를 출력할 때 출력할 데이터를 만드는 함수
    def __str__(self):
        return '질문: ' + self.question + '\n답변: ' + self.answer

In [3]:
# 75개의 대화 주제와 대화 주제에 따른 대화 주제 및 세부 대화 내용의 url을 파싱하는 함수
def getSubject():
    subjects = []     # 대화 주제를 기억하는 빈 리스트
    contentLinks = [] # 세부 대화 내용의 url을 기억하는 빈 리스트
    request = requests.get('https://basicenglishspeaking.com/daily-english-conversation-topics/')
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    divs = soup.findAll('div', {'class': 'tcb-col'})
    for div in divs:
        links = div.findAll('a')
        for link in links:
            # print(link.text)
            subjects.append(link.text)
            # print(link.get('href'))
            contentLinks.append(link.get('href'))
    return subjects, contentLinks

In [9]:
if __name__ == '__main__':
    subjects, contentLinks = getSubject()
    # for i in range(len(subjects)):
        # print('{0:2d}. {1} - {2}'.format(i + 1, subjects[i], contentLinks[i]))
    print('총 {}개의 대화를 찾았습니다.'.format(len(subjects)))
    
    # 대화 주제에 따른 모든 대화 내용을 저장할 빈 리스트를 선언한다.
    conversation = [] # Conversation 클래스 객체가 저장된다.
    
    i = 0 # 대화 주제 개수를 카운트 하는 변수
    # 대화 주제의 개수만큼 반복한다.
    for subject in subjects[:3]:
        # print('{0:2d} / {1} - {2}'.format(i + 1, len(subjects), subjects[i]))
        conversation.append('\n{0:2d} / {1} - {2}'.format(i + 1, len(subjects), subjects[i]))
        
        # 대화 주제별 대화 내용을 크롤링할 페이지를 요청한다.
        request = requests.get(contentLinks[i])
        
        # time 모듈을 import 하면 sleep() 메소드를 사용할 수 있다.
        # sleep() 메소드는 인수로 지정된 시간만큼 프로그램을 잠시 멈춘다. => 시간은 초 단위로 지정
        # 대형 포털 사이트는 짧은 간격으로 많은 요청이 들어오면 공격당하는 것으로 판단할 수 있기
        # 때문에 일정한 시간 간격을 두고 사이트의 정보를 요청해야 한다.
        time.sleep(1)
        
        html = request.text
        soup = BeautifulSoup(html, 'html.parser')
        # print(soup)
        
        # class 속성이 sc_player_container1인 모든 div 태그를 얻어온다.
        divs = soup.findAll('div', {'class': 'sc_player_container1'})
        # print(divs)
        # print(len(divs))
        
        # 각 대화 주제에 따른 대화 내용의 개수만큼 반복한다.
        for div in divs:
            # print(div)
            # index() : 객체의 인덱스 번호를 얻어온다.
            # 전체 대화 내용(divs)에서 현재 대화(div)의 인덱스를 얻어온다.
            # print(divs.index(div))
            
            if divs.index(div) % 2 == 0: # 질문인가?
                # div 태그의 인덱스가 짝수면 질문 변수에 질문을 저장한다.
                # next_sibling : 다음 형제
                # previous_sibling : 이전 형제
                question = div.next_sibling
            else:
                # div 태그의 인덱스가 홀수면 답변 변수에 답변을 저장한다.
                answer = div.next_sibling
                
                # 질문과 답변이 한 쌍이 되었으면 Conversation 클래스 객체를 생성하고 질문과 답변을 저장한다.
                c = Conversation(question, answer) # 대화 1건
                # 질문과 답변을 기억할 빈 리스트에 완성된 대화 1건(Conversation 클래스 객체)을 추가한다.
                conversation.append(c)
        
        # 대화 주제 1건이 처리 완료되었으므로 다음 대화를 처리하기 위해 i를 1증가 시킨다.
        i += 1
        
    for c in conversation:
        print(c)

총 75개의 대화를 찾았습니다.

 1 / 75 - Family
질문: How many people are there in your family?
답변: There are 5 people in my family: my father, mother, brother, sister, and me.
질문: Does your family live in a house or an apartment?
답변: We live in a house in the countryside.
질문: What does your father do?
답변: My father is a doctor. He works at the local hospital.
질문: How old is your mother?
답변: She is 40 years old, 1 year younger than my father.
질문: Do you have any siblings? What’s his/her name?
답변: Yes, I do. I have 1 elder brother, David, and 1 younger sister, Mary.
질문: Are you the oldest amongst your brothers and sisters?
답변: No, I’m not. I’m the second child in my family.
질문: What do your mother/father like?
답변: My father likes playing football and my mother likes cooking.
질문: Do your parents let you stay out late?
답변: Of course not. They always ask me to get home before 10 pm each night.
질문: Do you stay with your parents?
답변: Right now, no, but I used to.
질문: Does your family usually have dinner t